In [ ]:
from pathlib import Path

import numpy as np
import arviz as az
import pymc as pm
import pandas as pd
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from scipy.spatial import distance_matrix
from sklearn.cluster import KMeans

DATA_DIR = Path("../data/processed")
MODEL_DIR = Path("../models")

data = gpd.read_file(DATA_DIR / "spatial_income_1880.gpkg")
data["is_old"] = data.is_old.astype(int)
data = data.drop(index=data[data.population < 5].index).dropna().reset_index()
xy = np.array([data.geometry.x, data.geometry.y]).T
d = distance_matrix(xy, xy)
d_norm = d / d.max()
n = data.shape[0]

SEED = 42
N_CLUSTERS = len(data.group.unique())

In [ ]:
fig, (ax_1, ax_2, ax_3) = plt.subplots(
    3, 1, figsize=(8, 24), subplot_kw={"xticks": [], "yticks": []}
)
data.plot(
    column="lutheran",
    cmap="Reds",
    legend=True,
    ax=ax_1,
)
data.plot(
    column="orthodox_proportion",
    cmap="Reds",
    legend=True,
    ax=ax_2,
)
data.plot(
    column="total_income_ln",
    cmap="Reds",
    legend=True,
    ax=ax_3,
)
fig.tight_layout()
plt.show()

In [ ]:
with pm.Model() as model_1:
    W = pm.MutableData("W", data.total_income_ln)

    β_P = pm.MvNormal(
        "β_P", np.array([0, 0]), np.array(np.diagflat([0.1, 0.1])), shape=3
    )
    μ_P = β_P[0] + β_P[1] * W
    σ_P = pm.Exponential("σ_P", 1)
    P = pm.Normal("P", mu=μ_P, sigma=σ_P, observed=data.lutheran)

    β_O = pm.MvNormal(
        "β_O", np.array([0, 0]), np.array(np.diagflat([0.1, 0.1])), shape=3
    )

    μ_O = β_O[0] + β_O[1] * W
    σ_O = pm.Exponential("σ_O", 1)
    O = pm.Normal("O", mu=μ_O, sigma=σ_O, observed=data.orthodox_proportion)

    prior_1 = pm.sample_prior_predictive()
    posterior_1 = pm.sample(init="adapt_diag", return_inferencedata=True)
    posterior_prediction_1 = pm.sample_posterior_predictive(posterior_1)

In [ ]:
graph_1 = pm.model_to_graphviz(model_1)
graph_1

In [ ]:
# az.plot_posterior(posterior_1)
az.plot_trace(posterior_1)
az.plot_forest(posterior_1, combined=False, hdi_prob=0.94)

az.plot_ppc(posterior_prediction_1)

# Clustering

In [ ]:
cluster_colors = plt.get_cmap("tab20")
data.explore(
    column="group",
    categorical=True,
    cmap=cluster_colors,
    marker_kwds=dict(fill=True, radius=5),
    legend=True,
)

In [ ]:
data.plot(
    kind='scatter', x='total_income_ln', y='orthodox_proportion', c='group', cmap='tab20', figsize=(12, 12),
)
# for i in sorted(data.group.unique()):
    # data.loc[data.group == i, :].plot(kind='scatter', x='total_income_ln', y='orthodox_proportion')
    # plt.title(i)
    # plt.show()

# Final model with 12 geographical clusters (K-means)

$$ P_i \sim N(μ_{P,i}, σ_P), i = 0, 1, ... 846 $$
$$ O_i \sim N(μ_{O,i}, σ_O), i = 0, 1, ... 846 $$
$$ mi_{P,i} = [1 \; W]  β_{P,k[i]}, i = 0, 1, ... 846, k = 0, 1, ... 11 $$
$$ μ_{O,i} = [1 \; W]  β_{O,k[i]}, i = 0, 1, ... 846, k = 0, 1, ... 11 $$
$$ σ_P \sim Exp(1) $$
$$ σ_O \sim Exp(1) $$
$$ β_{P,k} \sim MvNormal \left( θ_P, \begin{bmatrix} 0.01 & 0 \\ 0 & 0.01 \end{bmatrix} \right), k = 0, 1, ... 11 $$
$$ β_{O,k} \sim MvNormal \left( θ_O, \begin{bmatrix} 0.01 & 0 \\ 0 & 0.01 \end{bmatrix} \right), k = 0, 1, ... 11 $$
$$ θ_{P} \sim MvNormal \left( \begin{bmatrix} 0 \\ 0 \end{bmatrix}, \begin{bmatrix} 0.1 & 0 \\ 0 & 0.1 \end{bmatrix} \right) $$
$$ θ_{O} \sim MvNormal \left( \begin{bmatrix} 0 \\ 0 \\ 0 \end{bmatrix}, \begin{bmatrix} 0.1 & 0 \\ 0 & 0.1 \end{bmatrix} \right) $$
where W is Component 3, i indicates the cadastral plot, and k[i] the cluster of the cadastral plot

In [ ]:
with pm.Model() as model_2:
    W = pm.MutableData("W", data.total_income_ln)
    idx = data.group

    θ_P = pm.Normal('θ_P', 100, 10)
    θ_O = pm.Normal('θ_O', 0.01, 10)

    # gaussian process?

    μ_P = pm.MvNormal(
        "μ_P", θ_P, R_P, shape=N_CLUSTERS,
    )
    μ_O = pm.MvNormal(
        "μ_O", θ_O, R_O, shape=N_CLUSTERS,
    )

    σ_P = pm.Exponential("σ_P", 1)
    P = pm.Normal("P", mu=μ_P, sigma=σ_P, observed=data.lutheran)

    σ_O = pm.Exponential("σ_O", 1)
    O = pm.Normal("O", mu=μ_O, sigma=σ_O, observed=data.orthodox_proportion_ln)

    prior_2 = pm.sample_prior_predictive(samples=100, random_seed=SEED)
    posterior_2 = pm.sample(
        draws=1000,
        tune=1000,
        init="adapt_diag",
        return_inferencedata=True,
        target_accept=0.9,
        random_seed=SEED,
    )
    posterior_prediction_2 = pm.sample_posterior_predictive(
        posterior_2, random_seed=SEED
    )

In [ ]:
graph_2 = pm.model_to_graphviz(model_2)
graph_2

## Prior predictive check

In [ ]:
prior = prior_2["prior"]
x = xr.DataArray(np.linspace(-5, 5, 100), dims=["x"])
y = prior["β_P"][:, :, 0, 0] + prior["β_P"][:, :, 0, 1] * x
_, ax = plt.subplots()
ax.plot(x, y.stack(sample=("chain", "draw")), c="k", alpha=0.1)
ax.set_xlabel("W")
ax.set_ylabel("μ_P")

In [ ]:
x = xr.DataArray(np.linspace(-5, 5, 100), dims=["x"])
y = prior["β_O"][:, :, 0, 0] + prior["β_O"][:, :, 0, 1] * x
_, ax = plt.subplots()
ax.plot(x, y.stack(sample=("chain", "draw")), c="k", alpha=0.1)
ax.set_xlabel("W")
ax.set_ylabel("μ_O")

## Posterior predictive check

In [ ]:
az.plot_ppc(posterior_prediction_2)

## Trace plots

In [ ]:
az.summary(posterior_2, hdi_prob=0.95)

In [ ]:
az.plot_posterior(posterior_2, var_names="β_O", grid=(12, 2))
az.plot_trace(posterior_2)
az.plot_forest(posterior_2, combined=True, hdi_prob=0.95)